In [ ]:
import os

os.environ['PYWIKIBOT_DIR'] = './wiki_reader/'

import sparknlp
from pyspark.sql import SQLContext

spark = sparknlp.start()
sc = spark.sparkContext
sqlCtx = SQLContext(sc)

In [ ]:
from ml.LogisticRegressionCached import readFromCache

(lrModel, pipelineModel) = readFromCache('./ml/train/')

def score_text(text):
    df = spark.createDataFrame([(text, 2)], ['text', 'target'])
    df_transformed = pipelineModel.transform(df) # To fix
    predictions = lrModel.transform(df_transformed)
    predictions = predictions.select(['text', 'probability', 'prediction'])
    pd_predictions = predictions.toPandas()
    positive_probability = pd_predictions.iloc[0]['probability'][1]
    overall_probability = 2 * positive_probability - 1
    
    return overall_probability

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import wiki_reader.reader as reader


wrapper = lambda r,b,l,cat,cont: reader.query(r,batch_size=b,limit=l,is_category=cat,preload_content=cont)
batch_size = 100
limit = None
preload_content = True
with ThreadPoolExecutor(max_workers=2) as e:
    e.submit(wrapper, 'peano', batch_size, limit, False, preload_content)